## Recurrent Neural Networks

In this assignment, we will learn about recurrent neural networks. We will create an RNN and learn to classify text data.

In [122]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import spacy
import nltk
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, SimpleRNN
from sklearn.model_selection import train_test_split


In [29]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [2]:
yelp = pd.read_csv('https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/yelp_labeled.csv', error_bad_lines=False)

b'Skipping line 281: expected 2 fields, saw 3\nSkipping line 290: expected 2 fields, saw 3\nSkipping line 296: expected 2 fields, saw 3\nSkipping line 322: expected 2 fields, saw 3\nSkipping line 373: expected 2 fields, saw 3\nSkipping line 417: expected 2 fields, saw 3\nSkipping line 427: expected 2 fields, saw 3\nSkipping line 429: expected 2 fields, saw 3\nSkipping line 577: expected 2 fields, saw 3\nSkipping line 578: expected 2 fields, saw 3\nSkipping line 611: expected 2 fields, saw 3\nSkipping line 677: expected 2 fields, saw 3\nSkipping line 771: expected 2 fields, saw 3\nSkipping line 930: expected 2 fields, saw 3\nSkipping line 979: expected 2 fields, saw 4\nSkipping line 980: expected 2 fields, saw 3\n'


In [3]:
yelp.head()

,text,sentiment
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [4]:
yelp.sentiment.value_counts()

1    494
0    482
Name: sentiment, dtype: int64

We have loaded a Yelp review dataset above. A positive sentiment is classified as 1 and a negative sentiment is classified as 0. 

In [5]:
from nltk.corpus import stopwords
import re
from nltk.stem.porter import PorterStemmer

stemmer = PorterStemmer()

def remove_stopwords(input_text):
    stopwords_list = stopwords.words('english')
    # Some words which might indicate a certain sentiment are kept via a whitelist
    whitelist = ["n't", "not", "no"]
    words = input_text.split() 
    clean_words = [word for word in words if (word not in stopwords_list or word in whitelist) and len(word) > 1] 
    return " ".join(clean_words)       

def stem_list(word_list):
    stemmed = []
    for word in word_list:
        stemmedword = stemmer.stem(word)
        stemmed.append(stemmedword)
    return stemmed

def normalize(terms):
    terms = terms.lower()
    terms = remove_stopwords(terms)
    word_delimiters = u'[\\[\\]\n.!?,;:\t\\-\\"\\(\\)\\\'\u2019\u2013 ]'
    term_list = re.split(word_delimiters, terms)
    trimmed = [x.rstrip() for x in term_list]
    stemmed = stem_list(trimmed)
    space = ' '
    normed = space.join(stemmed)
    normed = normed.replace('  ', ' ')
    return normed

In the code block above, we have functions to remove stopwords, stem, and normalize the text (remove special characters and trim white space). Apply the normalize function to every yelp review and assign the normalized text to a new column.

In [30]:
for a in yelp['text'][1].split():
  a = normalize(a)
  print(a)

crust

not
good 


In [51]:
# Answer below:

yelp['clean'] = ''
for i in range(yelp.shape[0]):
  temp = ''
  for word in yelp['text'][i].split():
    b = normalize(word)
    temp += (str(b)+' ')
  yelp['clean'][i] = temp

In [52]:
yelp['clean'].head()

0                                  wow   love  place  
1                                    crust  not good  
2                         not tasti   textur   nasti  
3    stop    late may bank holiday  rick steve reco...
4                     select   menu  great     price  
Name: clean, dtype: object

Next, use the one hot function for text encoding and encode the normalized text. Determine the vocabulary size to perform the encoding.

In [75]:
vocab = []
for i in range(yelp.shape[0]):
  temp = []
  for a in yelp['clean'][i].split():
    temp.append(a)
  vocab += temp

In [148]:
vocab = len(set(vocab))

In [150]:
# Answer below:
encoded = []
for i in range(yelp.shape[0]):
  temp = []
  for a in yelp['clean'][i].split():
    b = one_hot(a, vocab)
    temp.extend(b)
  encoded.append(temp)

In [151]:
yelp['clean'][0]

'wow   love  place  '

In [ ]:
encoded

Convert the encoded sequences into a numpy array and make sure all reviews are the same length using the `pad_sequences` function in Keras.

In [155]:
# Answer below:
features_array = pad_sequences(encoded)
features_array.shape

(976, 83)

In [156]:
maxw = np.max(features_array)
maxw

1627

Split the data into train and test. Use 20% for test. The sentiment column should be used as the target variable.

In [157]:
# Answer below:
X_train, X_test, y_train, y_test = train_test_split(features_array, yelp['sentiment'], test_size=0.2)

Create a sequential model. The model should contain an embedding layer with input dim that is the size of the largest encoding in the vocabulary. The output dim should be 100, the input length is the number of columns in the training data. 
After the embedding layer, add a SimpleRNN layer with unit size 32, a dense layer of size 8 and a dense output layer.

In [160]:
# Answer below:
model = Sequential()
model.add(Embedding(maxw+1, output_dim=100, input_length=features_array.shape[1]))
model.add(SimpleRNN(32))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 83, 100)           162800    
_________________________________________________________________
simple_rnn_5 (SimpleRNN)     (None, 32)                4256      
_________________________________________________________________
dense_10 (Dense)             (None, 8)                 264       
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 9         
Total params: 167,329
Trainable params: 167,329
Non-trainable params: 0
_________________________________________________________________


Compile using the optimizer of your choice, use crossentropy for your loss function. Fit the model using a batch size of 128 and 50 epochs

In [161]:
# Answer below:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size=128 , epochs=50 )

Epoch 1/50
7/7 [==============================] - 2s 106ms/step - loss: 0.6925 - accuracy: 0.5112 - val_loss: 0.6935 - val_accuracy: 0.4490
Epoch 2/50
7/7 [==============================] - 0s 39ms/step - loss: 0.6775 - accuracy: 0.7159 - val_loss: 0.6919 - val_accuracy: 0.4847
Epoch 3/50
7/7 [==============================] - 0s 38ms/step - loss: 0.6569 - accuracy: 0.8506 - val_loss: 0.6876 - val_accuracy: 0.5663
Epoch 4/50
7/7 [==============================] - 0s 37ms/step - loss: 0.6324 - accuracy: 0.8251 - val_loss: 0.6861 - val_accuracy: 0.5663
Epoch 5/50
7/7 [==============================] - 0s 38ms/step - loss: 0.6068 - accuracy: 0.7575 - val_loss: 0.6811 - val_accuracy: 0.5459
Epoch 6/50
7/7 [==============================] - 0s 37ms/step - loss: 0.5612 - accuracy: 0.8391 - val_loss: 0.7019 - val_accuracy: 0.5255
Epoch 7/50
7/7 [==============================] - 0s 37ms/step - loss: 0.5256 - accuracy: 0.8247 - val_loss: 0.6809 - val_accuracy: 0.5714
Epoch 8/50
7/7 [==========